## Imports

In [1]:
%matplotlib inline

from osgeo import osr, gdal

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import math

from PIL import Image, ImageDraw
import fiona

from copy import deepcopy

sns.set(color_codes=True)

## Read location of Roads

In [2]:
datafile = '../data/roads/kolovai-tonga_planet_osm_line_lines.shp'

possible_tags = set()

coll_list = []

with fiona.drivers():
    with fiona.open(datafile) as source:
        for coll in source:
            print(coll)
            coll_list.append(coll)

possible_tags

{'type': 'Feature', 'id': '0', 'geometry': {'type': 'LineString', 'coordinates': [(-175.33982444635888, -21.10495236721501), (-175.3398235, -21.1049178), (-175.3398188, -21.1048445), (-175.3398053, -21.1046368), (-175.3397978, -21.1045646), (-175.339776, -21.1043561), (-175.339768, -21.104306), (-175.3397564, -21.1042342), (-175.3397214, -21.103941), (-175.3397154, -21.1038961), (-175.3396651, -21.1035112), (-175.3396577, -21.1034614), (-175.3396442, -21.1033853), (-175.3396268, -21.103287), (-175.3395877, -21.1030623), (-175.3395592, -21.102898), (-175.3395548, -21.1028728), (-175.3395306, -21.1027333), (-175.3394113, -21.1021326), (-175.3393717, -21.1018988), (-175.339338, -21.1016779), (-175.3393221, -21.1015735), (-175.3392926, -21.1012682), (-175.3393043, -21.1010574), (-175.3393126, -21.1009074), (-175.3393205, -21.1006938), (-175.3393387, -21.1004127), (-175.3393556, -21.1001522), (-175.3393576, -21.10004), (-175.3393578, -21.1000243), (-175.3393612, -21.0998348), (-175.3393664,

set()

## Reading Image

In [3]:
datafile = gdal.Open("../data/aerial_image/kolovai.tif")
bnd1 = datafile.GetRasterBand(1).ReadAsArray()
bnd2 = datafile.GetRasterBand(2).ReadAsArray()
bnd3 = datafile.GetRasterBand(3).ReadAsArray()
nx = datafile.RasterXSize # Raster xsize
ny = datafile.RasterYSize # Raster ysize

img = np.dstack((bnd1, bnd2, bnd3))

In [4]:
img = Image.fromarray(img)

print(img.size)

(17761, 25006)


### Retrieve Latitude and Longitude of Image

In [5]:
old_cs= osr.SpatialReference()
old_cs.ImportFromWkt(datafile.GetProjectionRef())

# create the new coordinate system
wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
new_cs = osr.SpatialReference()
new_cs.ImportFromWkt(wgs84_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs, new_cs) 

#get the point to transform, pixel (0,0) in this case
W = datafile.RasterXSize
H = datafile.RasterYSize
gt = datafile.GetGeoTransform()
minx = gt[0]
miny = gt[3] + W*gt[4] + H*gt[5] 
maxx = gt[0] + W*gt[1] + H*gt[2]
maxy = gt[3]

#get the coordinates in lat long
xgeo0, ygeo0, _ = transform.TransformPoint(minx, miny)
xgeoN, ygeoN, _ = transform.TransformPoint(maxx, maxy)

In [6]:
print(xgeo0, ygeo0)
print(xgeoN, ygeoN)

-175.34907207131698 -21.104930856349313
-175.33536665317192 -21.08692801644621


## Process and Chunk

### Coordinate Converter

In [25]:
class Converter:
    def __init__(self, geo0, geoN, img_shape):
        self.xgeo0, self.ygeo0 = geo0
        self.xgeoN, self.ygeoN = geoN
        self.W, self.H = img_shape
        print(img_shape)
    
    def geo_2_pixel(self, x, y):
        x_pix = ((x - self.xgeo0)/abs(self.xgeoN - self.xgeo0)) * self.W
        y_pix = ((y - self.ygeo0)/abs(self.ygeoN - self.ygeo0)) * self.H
        
        return int(round(x_pix)), int(round(y_pix))

def world2Pixel(geoMatrix, x, y):
    """
    Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
    the pixel location of a geospatial coordinate 
    """
    ulX = geoMatrix[0]
    ulY = geoMatrix[3]
    xDist = geoMatrix[1]
    yDist = geoMatrix[5]
    rtnX = geoMatrix[2]
    rtnY = geoMatrix[4]
    pixel = int((x - ulX) / xDist)
    line = int((ulY - y) / yDist)
    
    return (pixel, line)

transform2 = osr.CoordinateTransformation(new_cs, old_cs) 

### Mark roads on copy of original image

In [22]:
loc_converter = Converter((xgeo0, ygeo0), (xgeoN, ygeoN), img.size)

marked_img = deepcopy(img)

draw_zone = ImageDraw.Draw(marked_img)

(17761, 25006)


In [29]:
road = []
y_max = 0
x_max = 0
x_min, y_min = marked_img.size
for loc in coll_list[0]['geometry']['coordinates']:
#     x, y = loc_converter.geo_2_pixel(loc[0], loc[1])
    x_i, y_i, _ = transform.TransformPoint(loc[0], loc[1])
    x, y = world2Pixel(gt, x_i, y_i)
    if x < 0 or y < 0:
        continue
    road.append((x, y))
    y_max = max(y_max, y)
    x_max = max(x_max, x)
    x_min = min(x_min, x)
    y_min = min(y_min, y)
    
print(x_min, y_min)
print(x_max, y_max)

Image.MAX_IMAGE_PIXELS = None
    
draw_zone.polygon(road, outline=(255, 0, 0, 255), fill=(255, 0, 0, 255))
marked_img.crop((x_min, y_min, x_max, y_max)).show()

17761 25006
227236764 27965432


MemoryError: 

### Slice marked image

In [33]:
img_w, img_h = marked_img.size

slice_x = 1000
slice_y = 1000

Image.MAX_IMAGE_PIXELS = None

max_y = int(math.ceil(img_h/slice_y))
max_x = int(math.ceil(img_w/slice_x))


for y_start in range(0, max_y):
    for x_start in range(0, max_x):
        left = slice_x * x_start
        upper = slice_y * y_start
        bbox = (left, upper, left+slice_x, upper+slice_y)
        working_slice = marked_img.crop(bbox)
        
        working_slice.save("../data/aerial_image/slices/marked__x_"+str(left)+"-"+str(left+slice_x)+"__y_"+ \
                           str(upper)+"-"+str(upper+slice_y)+".png")